In [1]:
from contrans import contrans
import numpy as np # type: ignore
import pandas as pd # type: ignore

In [ ]:
ct = contrans()
ct.congresskey

In [ ]:
useragent = ct.get_useragent()
useragent

In [ ]:
headers = ct.make_headers()
headers

In [ ]:
bio_df = ct.get_bioguideIDs()
bio_df

In [ ]:
bioguides =ct.get_bioguideIDs()
bio_df = pd.json_normalize(bioguides, record_path=['members'])
bio_df[['name', 'state', 'district', 'bioguideId']]

In [ ]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr', 'party_code']]
members

In [ ]:
votes = ct.get_votes()
votes

In [ ]:
house = house[['rollnumber','icpsr', 'cast_code']]

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

In [ ]:
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree':'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
agreement = pd.merge(house_mat, members,
                     left_on = 'icpsr_x',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icpsr_x'], axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icpsr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1', 'party1', 'member2', 'party2', 'agree']]
agreement

In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree',  ascending=False).head(30)